<a href="https://colab.research.google.com/github/upayuryeva/Social-Network-Analysis-Project/blob/main/SN_project_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>Social Network Analysis Project
<center><h3>Irina Yuryeva

Friend list was collected from my vk page: [Ira Yuryeva](https://vk.com/upayuryeva)

#Collect data from VK 

In [ ]:
!pip install vk_api
import vk_api

In [ ]:
def auth_handler():
    """ Function for accounts with two factor authentication 
    """

    # Two factor authentication code
    key = input("Enter authentication code: ")
    remember_device = True

    return key, remember_device

In [ ]:
email = 'YOUR EMAIL OR PHONE NUMBER'
password = 'YOUR PASSWORD'
vk_session = vk_api.VkApi(email, password, auth_handler=auth_handler)
vk_session.auth()

vk = vk_session.get_api()

Collecting my friends list

In [ ]:
def get_friends_list(user_id=None,
                    fields_to_return=['sex', 'country','education','city', 'bdate']):
  
    """
    Gets friends list with specific fields, removes deleted pages 
    and selects and changes only important fields to make more readable
    dictionaries of information of every friend

    Parameters
    ----------
      user_id: int

      fields_to_return: list of strs
        Profile fields to return. 
        Sample values: nickname, domain, sex, bdate, city, country, timezone, 
        photo_50, photo_100, photo_200_orig, has_mobile, contacts, education, 
        online, relation, last_seen, status, can_write_private_message, 
        can_see_all_posts, can_post, universities. 

    Returns
    -------
      list of dicts of information about friends
    """

    #collect friends list
    raw_list = vk.friends.get(user_id=user_id, fields=fields_to_return)
    
    #friends list preparation
    friends_list = []

    for friend in raw_list['items']:
      if friend.get('deactivated') != 'deleted':
        friend_new = {}

        if friend['sex'] == 1:
          sex = 'ж'
        else:
          sex = 'м'

        bdate = friend.get('bdate', '')
        if len(bdate) > 6:
          byear = bdate[len(bdate)-4:]
        else:
          byear = ''

        friend_new.update({'name' : friend['first_name'] + ' ' + friend['last_name'],
                           'name_label': friend['first_name'] + '\n' + friend['last_name'],
                           'sex': sex, 
                           'byear': byear,
                           'id' : friend['id'],
                           'city': friend.get('city', {}).get('title', ''),
                           'country': friend.get('country', {}).get('title', ''),
                           'faculty_name': friend.get('faculty_name', ''), 
                           'university_name': friend.get('university_name', '')
                           })
        friends_list.append(friend_new)
      
    return friends_list

In [ ]:
my_friends_list = get_friends_list()
my_friends_names = [friend['name'] for friend in my_friends_list]
my_friends_list[23]

{'byear': '',
 'city': 'Москва',
 'country': 'Россия',
 'faculty_name': 'Физический факультет',
 'id': 26663918,
 'name': 'Светик Пантелеева',
 'name_label': 'Светик\nПантелеева',
 'sex': 'ж',
 'university_name': 'МГУ'}

Finging friends connections to make edges for our graph and save file with these edges

In [ ]:
from google.colab import files

#collect friends of friends
def save_friends_edges(main_friends_list):
    """
    Collects friends of friends of main user, makes edges for graph of friends 
    connections, find number of mutual friends to set edges attributes, 
    save these edges and edges attributes to .txt files, find number of friends
    and update main_friends_list to make new node attribute

    Start this function only if you're not already have file with edges

    Parameters
    ----------
      main_friends_list: list of dicts
        list of dicts of information about friends

    Returns
    -------
      saves files with edges and edges attributes
    """
    friends_connections = [] # list of tuples
    main_friends_ids = [friend['id'] for friend in main_friends_list]
    list_num_mutfr = [] # for edge attributes 
    main_friends_list_new = [] #with new attribute number of friends

    for friend in main_friends_list:
      this_friend_friends = get_friends_list(user_id=friend['id'])
      for friend_friend in this_friend_friends:
        edge = tuple(sorted([friend['name'], friend_friend['name']]))
        if (friend_friend['id'] in main_friends_ids and 
            edge not in friends_connections and 
            friend_friend['name'] != 'Ира\nЮрьева'):
          friends_connections.append(edge)
          list_num_mutfr.append(len(vk.friends.getMutual(source_uid=friend['id'],
                                                         target_uid=friend_friend['id'])))
      friend.update({'n_friends': len(this_friend_friends)})
      main_friends_list_new.append(friend)
    
    main_friends_list = main_friends_list_new

    with open('friends_edges.txt', 'w') as f:
        f.write('\n'.join('%s %s' % x for x in friends_connections))

    with open('friends_edges_attributes.txt', 'w') as f:
        f.write('\n'.join('%s' % x for x in list_num_mutfr))

In [ ]:
file_with_edges = False #change to True if you already have files 

if not file_with_edges:
  save_friends_edges(my_friends_list)

In [ ]:
my_friends_list[11]

{'byear': '1989',
 'city': 'Новосибирск',
 'country': 'Россия',
 'faculty_name': '',
 'id': 11581630,
 'n_friends': 1725,
 'name': 'Светлана Вечкапова',
 'name_label': 'Светлана\nВечкапова',
 'sex': 'ж',
 'university_name': ''}



---

Open file

In [ ]:
with open('friends_edges.txt') as f:
  raw_edges = f.readlines()

with open('friends_edges_attributes.txt') as f:
  edges_attrs = f.readlines()

In [ ]:
edges = []

for r_edge in raw_edges:
  line = r_edge.split(' ')
  edge = (line[0] + ' ' + line[1], line[2] + ' ' + line[3][:len(line[3])-1])
  edges.append(edge)

edges_attrs = [int(attr) for attr in edges_attrs]


In [ ]:
edges[:10]

[('Елена Григорьева', 'Елена Кальнеус'),
 ('Анна Самосекова', 'Елена Кальнеус'),
 ('Елена Кальнеус', 'Татьяна Анискова'),
 ('Елена Кальнеус', 'Мария Попова'),
 ('Артём Романов', 'Елена Кальнеус'),
 ('Артем Шеболтасов', 'Елена Кальнеус'),
 ('Наталья Загорулько', 'Юлия Юрьева'),
 ('Дмитрий Овечкин', 'Наталья Загорулько'),
 ('Елена Григорьева', 'Марина Волкова'),
 ('Анна Самосекова', 'Елена Григорьева')]

Make graph of the resulting edges

In [ ]:
import networkx as nx

In [ ]:
edges_attributes = [{'mut_friends': num} for num in edges_attrs]
edges_attributes = zip(edges, edges_attributes)
edges_attributes = dict(edges_attributes)

In [ ]:
friends_attributes = zip(my_friends_names, my_friends_list)
friends_attributes = dict(friends_attributes)

In [ ]:
G = nx.Graph()
G.add_nodes_from(friends_attributes)
nx.set_node_attributes(G, friends_attributes)
G.add_edges_from(edges)
nx.set_edge_attributes(G, edges_attributes)
list(G.nodes())[:10]

['Елена Кальнеус',
 'Марина Юрьева',
 'Наталья Загорулько',
 'Елена Григорьева',
 'Марина Волкова',
 'Мария Бармасова',
 'Алёна Захрямина',
 'Александр Хижик',
 'Анна Самосекова',
 'Антон Наумов']

In [ ]:
len(G.nodes())

159

In [ ]:
G.nodes['Марина Юрьева']

{'byear': '1986',
 'city': 'Hartford',
 'country': 'США',
 'faculty_name': 'Факультет биоинженерии и биоинформатики',
 'id': 2233480,
 'n_friends': 121,
 'name': 'Марина Юрьева',
 'name_label': 'Марина\nЮрьева',
 'sex': 'ж',
 'university_name': 'МГУ'}

In [ ]:
G['Надежда Рухляда']['Ольга Апальчук']['mut_friends']

23

In [ ]:
nx.write_gexf(G, "friends_graph.gexf")